## 13. 机器人的运动范围

地上有一个 rows 行和 cols 列的方格。坐标从 [0,0] 到 [rows-1,cols-1] 。一个机器人从坐标 [0,0] 的格子开始移动，每一次只能向左，右，上，下四个方向移动一格，但是不能进入行坐标和列坐标的数位之和大于 threshold 的格子。 例如，当 threshold 为 18 时，机器人能够进入方格   [35,37] ，因为 3+5+3+7 = 18。但是，它不能进入方格 [35,38] ，因为 3+5+3+8 = 19 。请问该机器人能够达到多少个格子？

数据范围：$0≤threshold≤15$, $1≤rows,cols≤100$

进阶：空间复杂度 $O(nm)$, 时间复杂度 $O(nm)$

方法一

深度优先搜索（dfs）

深度优先搜索一般用于树或者图的遍历，其他有分支的（如二维矩阵）也适用。它的原理是从初始点开始，一直沿着同一个分支遍历，直到该分支结束，然后回溯到上一级继续沿着一个分支走到底，如此往复，直到所有的节点都有被访问到。

思路：

我们从[0,0]开始，每次选择一个方向开始检查能否访问，如果能访问进入该节点，该节点作为子问题，继续按照这个思路访问，一条路走到黑，然后再回溯，回溯的过程中每个子问题再选择其他方向，正是深度优先搜索。

具体做法：

1. 检查若是 threshold 小于等于0，只能访问起点这个格子。
2. 从起点开始深度优先搜索，每次访问一个格子的下标时，检查其有没有超过边界，是否被访问过了。同时用连除法分别计算每个下标的位数和，检查位数和是否大于threshold。
3. 若是都符合访问条件，则进行访问，增加访问的格子数，同时数组中标记为访问过了。
4. 然后遍历四个方向，依次进入四个分支继续访问。

In [ ]:
class Solution:
    def __init__(self):
        #记录遍历的四个方向
        self.dir = [[-1, 0], [1, 0], [0, -1], [0, 1]]
        #记录答案
        self.res = 0
        
    #计算一个数字的每个数之和
    def cal(self, n: int) -> int:
        sum = 0
        #连除法算出每一位
        while n:
            sum += (n % 10)
            n //= 10
        return sum
    
    #深度优先搜索dfs
    def dfs(self, i: int, j: int, rows: int, cols: int, threshold: int, vis: List[List[bool]]):
        #越界或者已经访问过
        if i < 0 or i >= rows or j < 0 or j >= cols or not vis[i][j]:
            return
        #行列和数字相加大于threshold，不可取
        if self.cal(i) + self.cal(j) > threshold:
            return
        self.res += 1
        #标记经过的位置
        vis[i][j] = False
        #上下左右四个方向搜索
        for k in range(4):
            self.dfs(i + self.dir[k][0], j + self.dir[k][1], rows, cols, threshold, vis)
    
    def movingCount(self , threshold: int, rows: int, cols: int) -> int:
        #判断特殊情况
        if threshold <= 0:
            return 1
        #标记某个格子没有被访问过
        vis = [[True for i in range(cols)] for j in range(rows)]
        self.dfs(0, 0, rows, cols, threshold, vis)
        return self.res


方法二

深度优先搜索是一条路走到底，我们还可以尝试广度优先搜索。只要遍历的时候从起点开始，依次访问与其相邻的四个方向（如果可以访问），然后再将与这四个方向相邻的节点都依次加入队列中排队等待访问。这样下来，每次访问都是一圈一圈推进，因此是广度优先搜索。

具体做法：

1. 检查若是threshold小于等于0，只能访问起点这个格子。
2. 从起点开始广度优先搜索，将其优先加入队列中等待访问，队列记录要访问的坐标。
3. 每次取出队列首部元素访问，进行标记，然后计数。
4. 然后遍历四个方向相邻的格子，检查其有没有超过边界，是否被访问过了。同时用连除法分别计算每个下标的位数和，检查位数和是否大于threshold，如果满足可以访问的条件，则将其加入队列中等待访问，同时标记访问过了。

In [ ]:
class Solution:
    def __init__(self):
        #记录遍历的四个方向
        self.dir = [[-1, 0], [1, 0], [0, -1], [0, 1]]
        
    #计算一个数字的每个数之和
    def cal(self, n: int) -> int:
        sum = 0
        #连除法算出每一位
        while n:
            sum += (n % 10)
            n //= 10
        return sum
    
    def movingCount(self , threshold: int, rows: int, cols: int) -> int:
        #判断特殊情况
        if threshold <= 0:
            return 1
        #标记某个格子没有被访问过
        vis = [[True for i in range(cols)] for j in range(rows)]
        #记录答案
        res = 0
        #起点先入队列
        q = [[0, 0]]
        vis[0][0] = False
        while len(q):
            #获取符合条件的一步格子
            node = q[0]
            q.pop(0)
            res += 1
            #遍历四个方向
            for i in range(4):
                x = node[0] + self.dir[i][0]
                y = node[1] + self.dir[i][1]
                #符合条件的下一步才进入队列
                if x >= 0 and x < rows and y >= 0 and y < cols and vis[x][y]:
                    if self.cal(x) + self.cal(y) <= threshold:
                        q.append([x, y])
                        vis[x][y] = False
        return res
